**Import essential library**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier

import sklearn
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import warnings
warnings.simplefilter("ignore")

**Load dataset**

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
# Load dataset
def load_data():
    data =pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/processing_file2.csv", engine='python')
    return data

In [3]:
tweet_df = load_data()
tweet_df.head()

,Unnamed: 0,UserName,label,Tweet_stemmed,Tweet_lemmatized
0,0,3799,0.0,"['menyrbi', 'philgahan', 'chrisitv', 'httpstco...","['menyrbie', 'philgahan', 'chrisitv', 'httpstc..."
1,1,3800,1.0,"['advic', 'talk', 'neighbour', 'famili', 'exch...","['advice', 'talk', 'neighbour', 'family', 'exc..."
2,2,3801,1.0,"['coronaviru', 'australia', 'woolworth', 'give...","['coronavirus', 'australia', 'woolworth', 'giv..."
3,3,3802,1.0,"['food', 'stock', 'one', 'empti', 'pleas', 'do...","['food', 'stock', 'one', 'empty', 'please', 'd..."
4,4,3803,3.0,"['readi', 'go', 'supermarket', 'covid', 'outbr...","['ready', 'go', 'supermarket', 'covid', 'outbr..."


In [4]:
tweet_df.drop('Unnamed: 0',inplace=True,axis = 1)

In [5]:
tweet_df.head()

,UserName,label,Tweet_stemmed,Tweet_lemmatized
0,3799,0.0,"['menyrbi', 'philgahan', 'chrisitv', 'httpstco...","['menyrbie', 'philgahan', 'chrisitv', 'httpstc..."
1,3800,1.0,"['advic', 'talk', 'neighbour', 'famili', 'exch...","['advice', 'talk', 'neighbour', 'family', 'exc..."
2,3801,1.0,"['coronaviru', 'australia', 'woolworth', 'give...","['coronavirus', 'australia', 'woolworth', 'giv..."
3,3802,1.0,"['food', 'stock', 'one', 'empti', 'pleas', 'do...","['food', 'stock', 'one', 'empty', 'please', 'd..."
4,3803,3.0,"['readi', 'go', 'supermarket', 'covid', 'outbr...","['ready', 'go', 'supermarket', 'covid', 'outbr..."


In [6]:
tweet_df.drop('UserName',inplace=True,axis = 1)

In [7]:
tweet_df.head()

,label,Tweet_stemmed,Tweet_lemmatized
0,0.0,"['menyrbi', 'philgahan', 'chrisitv', 'httpstco...","['menyrbie', 'philgahan', 'chrisitv', 'httpstc..."
1,1.0,"['advic', 'talk', 'neighbour', 'famili', 'exch...","['advice', 'talk', 'neighbour', 'family', 'exc..."
2,1.0,"['coronaviru', 'australia', 'woolworth', 'give...","['coronavirus', 'australia', 'woolworth', 'giv..."
3,1.0,"['food', 'stock', 'one', 'empti', 'pleas', 'do...","['food', 'stock', 'one', 'empty', 'please', 'd..."
4,3.0,"['readi', 'go', 'supermarket', 'covid', 'outbr...","['ready', 'go', 'supermarket', 'covid', 'outbr..."


In [8]:
tweet_df.isnull().sum()

label               0
Tweet_stemmed       0
Tweet_lemmatized    0
dtype: int64

In [9]:
messages= tweet_df.copy()

In [10]:
messages.reset_index()

,index,label,Tweet_stemmed,Tweet_lemmatized
0,0,0.0,"['menyrbi', 'philgahan', 'chrisitv', 'httpstco...","['menyrbie', 'philgahan', 'chrisitv', 'httpstc..."
1,1,1.0,"['advic', 'talk', 'neighbour', 'famili', 'exch...","['advice', 'talk', 'neighbour', 'family', 'exc..."
2,2,1.0,"['coronaviru', 'australia', 'woolworth', 'give...","['coronavirus', 'australia', 'woolworth', 'giv..."
3,3,1.0,"['food', 'stock', 'one', 'empti', 'pleas', 'do...","['food', 'stock', 'one', 'empty', 'please', 'd..."
4,4,3.0,"['readi', 'go', 'supermarket', 'covid', 'outbr...","['ready', 'go', 'supermarket', 'covid', 'outbr..."
...,...,...,...,...
31231,31231,2.0,"['mrsilverscott', 'definit', 'man', 'feel', 'l...","['mrsilverscott', 'definitely', 'man', 'feel',..."
31232,31232,0.0,"['airlin', 'pilot', 'offer', 'stock', 'superma...","['airline', 'pilot', 'offering', 'stock', 'sup..."
31233,31233,3.0,"['respons', 'complaint', 'provid', 'cite', 'co...","['response', 'complaint', 'provided', 'citing'..."
31234,31234,1.0,"['know', 'get', 'tough', 'kameronwild', 'ratio...","['know', 'getting', 'tough', 'kameronwilds', '..."


In [11]:
# For triagram
tfidf_triagram = TfidfVectorizer(sublinear_tf=True,
                            analyzer='word',
                            ngram_range=(3, 3),
                            max_features=8000)

In [12]:
X_tiagram = tfidf_triagram.fit_transform(messages['Tweet_lemmatized'])

In [13]:
Y_tiagram = messages['label']

In [14]:
smk_tiagram = SMOTETomek(random_state=42)
X_tiagram_v1,Y_tiagram_v1=smk_tiagram.fit_sample(X_tiagram,Y_tiagram)

In [15]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(Y_tiagram)))
print('Resampled dataset shape {}'.format(Counter(Y_tiagram_v1)))

Original dataset shape Counter({1.0: 11421, 0.0: 7711, 2.0: 6624, 3.0: 5480})
Resampled dataset shape Counter({3.0: 11253, 0.0: 11185, 2.0: 11178, 1.0: 11126})


In [16]:
# data Split
# Biagram Model
# let's see how well our model perform on this new data
x_train_tiagram,x_test_tiagram,y_train_tiagram,y_test_tiagram = train_test_split(X_tiagram_v1,Y_tiagram_v1,test_size = 0.30, random_state= True)

In [18]:
import matplotlib.pyplot as plt
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [25]:
# Decission Tree Classifier
decession_tree_tiagram = DecisionTreeClassifier()
decession_tree_tiagram.fit(x_train_tiagram,y_train_tiagram)
y_pred_decession_tree_tiagram = decession_tree_tiagram.predict(x_test_tiagram)
print("Decission Tree Classifier")
print(classification_report(y_test_tiagram, y_pred_decession_tree_tiagram))


# Random Forest  Classifier
random_forest_tiagram = RandomForestClassifier()
random_forest_tiagram.fit(x_train_tiagram,y_train_tiagram)
y_pred_random_forest_tiagram = random_forest_tiagram.predict(x_test_tiagram)
print("Random Forest  Classifier")
print(classification_report(y_test_tiagram, y_pred_random_forest_tiagram))


# KNN  Algo
knn_classifier_tiagram = KNeighborsClassifier()
knn_classifier_tiagram.fit(x_train_tiagram,y_train_tiagram)
y_pred_knn_classifier_tigram = knn_classifier_tiagram.predict(x_test_tiagram)
print("KNN  Algo")
print(classification_report(y_test_tiagram, y_pred_knn_classifier_tigram))


# Multinomial Naive Bias
mulNB_tiagram  = MultinomialNB()
mulNB_tiagram.fit(x_train_tiagram,y_train_tiagram)
y_pred_mulNB_tiagram = mulNB_tiagram.predict(x_test_tiagram)
print("Multinomial Naive Bias")
print(classification_report(y_test_tiagram, y_pred_mulNB_tiagram))


# Gradient bossting
gradient_bossting_tiagram  = GradientBoostingClassifier(random_state=3)
gradient_bossting_tiagram.fit(x_train_tiagram,y_train_tiagram)
y_pred_gradientbossting = gradient_bossting_tiagram.predict(x_test_tiagram)
print("GradientBosting  Algo")
print(classification_report(y_test_tiagram, y_pred_gradientbossting))


# Logistic Regressor
logis_reg_tia =LogisticRegression()
logis_reg_tia.fit(x_train_tiagram,y_train_tiagram)
y_pred_log = logis_reg_tia.predict(x_test_tiagram)
print("Logistic Reg  Algo")
print(classification_report(y_test_tiagram, y_pred_log))

Random Forest  Classifier
              precision    recall  f1-score   support

         0.0       0.33      0.82      0.47      3358
         1.0       0.49      0.21      0.29      3377
         2.0       0.58      0.34      0.43      3275
         3.0       0.70      0.37      0.49      3413

    accuracy                           0.43     13423
   macro avg       0.53      0.43      0.42     13423
weighted avg       0.53      0.43      0.42     13423

KNN  Algo
              precision    recall  f1-score   support

         0.0       0.43      0.30      0.35      3358
         1.0       0.36      0.13      0.19      3377
         2.0       0.29      0.73      0.42      3275
         3.0       0.63      0.30      0.41      3413

    accuracy                           0.36     13423
   macro avg       0.43      0.37      0.34     13423
weighted avg       0.43      0.36      0.34     13423

Multinomial Naive Bias
              precision    recall  f1-score   support

         0.0    

In [27]:
svm_class_tiagram = svm.SVC(kernel='poly')
svm_class_tiagram.fit(x_train_tiagram,y_train_tiagram)
y_pred_svm = svm_class_tiagram.predict(x_test_tiagram)
print("SVM  Algo")
print(classification_report(y_test_tiagram, y_pred_svm))



# Logistic Regressor
logis_reg_tia =LogisticRegression()
logis_reg_tia.fit(x_train_tiagram,y_train_tiagram)
y_pred_log = logis_reg_tia.predict(x_test_tiagram)
print("Logistic Reg  Algo")
print(classification_report(y_test_tiagram, y_pred_log))

SVM  Algo
              precision    recall  f1-score   support

         0.0       0.31      0.87      0.46      3358
         1.0       0.54      0.13      0.21      3377
         2.0       0.63      0.28      0.38      3275
         3.0       0.69      0.34      0.46      3413

    accuracy                           0.41     13423
   macro avg       0.54      0.41      0.38     13423
weighted avg       0.54      0.41      0.38     13423

Logistic Reg  Algo
              precision    recall  f1-score   support

         0.0       0.33      0.81      0.47      3358
         1.0       0.46      0.19      0.27      3377
         2.0       0.56      0.34      0.42      3275
         3.0       0.70      0.38      0.49      3413

    accuracy                           0.43     13423
   macro avg       0.51      0.43      0.41     13423
weighted avg       0.52      0.43      0.41     13423

